In [126]:
import wandb
import matplotlib.pyplot as plt
import scipy
import numpy as np
import json
from IPython.display import clear_output

plt.rcParams["axes.grid"] = False

sys.path.append('..')

In [127]:
language = 'french'

sys.argv = [
"--device cuda",
"--data-folder", "..\\data",
"--seed", "13",
"--configuration", "rnn-simple",
"--language", language,
"--challenge", "named-entity-recognition"]

In [128]:
# Configure container:
from dependency_injection.ioc_container import IocContainer

container = IocContainer()

In [129]:
entity = 'eval-historical-texts'
project = 'named-entity-recognition'

In [130]:
def get_wandb_runs():
    api = wandb.Api()
    runs = api.runs(path=f'{entity}/{project}', filters={
        'createdAt': {
            '$gt': '20200621000000'
        },
        'config.language': language,
        'state': {
            '$ne': 'running'
        }
    })

    return runs

In [131]:
runs = get_wandb_runs()
print(f'Loaded {len(runs)} runs')

Loaded 240 runs


In [132]:
def get_summary_value(run, key: str):
    if key not in run.summary.keys():
        return None

    return run.summary[key]

pretrained_model_type_key = 'pretrained_model_type'
fine_tune_key = 'fine_tune'

for i, run in enumerate(runs):
    if 'pretrained_model_type' in run.config.keys():
        continue
 
    clear_output(wait=True)
    print(f'Updating... ({i}/{len(runs)})')
    if len(run.config.keys()) == 0:# or fine_tune_key in run.config.keys() or pretrained_model_type_key in run.config.keys():
        continue

    if 'pretrained_model_type' in run.config.keys():
        continue

    if 'pretrained_model_type' not in run.config.keys():
        include_pretrained_model = run.config['include_pretrained_model']
        include_fasttext_model = run.config['include_fasttext_model']
        fine_tune_pretrained = run.config['fine_tune_pretrained']
        fine_tune_after_convergence = run.config['fine_tune_after_convergence']

        pretr_type = 'none'
        if include_pretrained_model and include_fasttext_model:
            pretr_type = 'both'
        elif include_pretrained_model:
            pretr_type = 'bert'
        elif include_fasttext_model:
            pretr_type = 'fast-text'

        fine_tune = (fine_tune_pretrained or fine_tune_after_convergence)
        
        print('Updating configuration...')
        run.config.update({
            'pretrained_model_type': pretr_type,
            'fine_tune': fine_tune
        })


    all_scores = [
        get_summary_value(run, 'Best - f1-score-micro-partial-all-component'),
        get_summary_value(run, 'Best - f1-score-micro-partial-all-literal-coarse'),
        get_summary_value(run, 'Best - f1-score-micro-partial-all-literal-fine'),
        get_summary_value(run, 'Best - f1-score-micro-partial-all-metonymic-coarse'),
        get_summary_value(run, 'Best - f1-score-micro-partial-all-metonymic-fine'),
        get_summary_value(run, 'Best - f1-score-micro-partial-all-nested')
    ]

    all_scores = [x for x in all_scores if x is not None]
    avg_score = np.mean(all_scores)
        
    print('Updating summary...')
    run.summary.update({
        'average_score': avg_score
    })

    run.update()

Updating... (16/240)
Updating configuration...
Updating summary...
